In [1]:
#Loads the data

import pandas as pd
df = pd.read_excel("Online Retail.xlsx") 
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Data Preprocessing
Data Preprocessing helps us in transforming the data according to our need. The invalid data like missing values and null values need to be removed for the ease of processing.

So, let's perform this to your data.

Since the Customer Id is the primary attribute for the customer segmentation, we remove the null values. This snippet helps in removing the duplicates and null values if present. *italicized text*

In [0]:
customer_country=df[['Country','CustomerID']].drop_duplicates()

df= df[pd.notnull(df['CustomerID'])]

df = df[(df['Quantity']>0)]

### Total Price is needed for the calculation of monetary value.

In [0]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## Date Format
It would be clumsy if the date format doesn't match with our need. Let's design it to the way we want.

In [0]:
#Changing date format 

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [0]:
#recency calculation till the maximum date of the dataset (9/12/2011)

import datetime as dt

NOW = dt.datetime(2011,12,10)

In [0]:
rfmTable=df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})

In [0]:
rfmTable.rename(columns={'InvoiceDate': 'recency', 'InvoiceNo': 'frequency', 'TotalPrice': 'monetary_value'}, inplace=True)

In [11]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,1,77183.60
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,18,73,1757.55
12350.0,310,17,334.40


### RFM Ranking
Now, its time to rank our customers.

Let's rank them from 1 to 4. For our easy calculation lets normalize the data with the quantiles. In this case, we are going to divide the data into four quantiles.

In [0]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

quantiles = quantiles.to_dict()

In [0]:
rfm = rfmTable

def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [0]:
#R score calculation
rfm['r_quartile'] =rfm['recency'].apply(RScore, args=('recency',quantiles,))  
 

In [23]:
rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile
CustomerID,,,,,
12346.0,325,1,77183.60,4,4
12347.0,2,182,4310.00,1,1
12348.0,75,31,1797.24,3,3
12349.0,18,73,1757.55,2,2
12350.0,310,17,334.40,4,4


In [0]:
 def  FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [0]:
rfm['f_quartile'] = rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
    


In [0]:
rfm['m_quartile'] =rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

In [27]:
#combine RFM score
 
rfm['RFMScore'] = rfm.r_quartile.map(str)+ rfm.f_quartile.map(str)+ rfm.m_quartile.map(str)
 
rfm[rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
14646.0,1,2080,280206.02,1,1,1,111
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
14911.0,1,5677,143825.06,1,1,1,111
14156.0,9,1400,117379.63,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
